In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

from sklearn.pipeline import Pipeline

from nltk.stem import SnowballStemmer
import spacy
import re, unicodedata

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from num2words import num2words

import swifter

from joblib import dump, load

In [2]:
data = pd.read_excel("data/cat_345.xlsx")

In [3]:
data.sample(3)

,Textos_espanol,sdg
2089,"Sostiene que las normas que afectan a los acuerdos privados de mujeres y hombres tienen un fuerte impacto en la participaciÃ³n de las mujeres en la vida pÃºblica. El capÃtulo contiene cuatro secciones principales: derecho de familia, integridad fÃsica, nacionalidad y acceso a la justicia. Cada secciÃ³n destaca los respectivos puntos de referencia internacionales, las tendencias en la legislaciÃ³n nacional y las lagunas de cumplimiento en la regiÃ³n de Oriente Medio y Norte de Ãfrica. En el Ã¡mbito del derecho de familia y la nacionalidad, el capÃtulo ofrece una visiÃ³n general de las discriminaciones legales que siguen obstaculizando la igualdad de la mujer con el hombre. En cuanto al acceso a la justicia y la violencia de gÃ©nero, muchos paÃses reconocen las cuestiones pendientes y se proponen abordar las lagunas restantes mediante agendas nacionales de reforma. Sin embargo, es necesario seguir actuando para permitir un acceso igualitario a la justicia y eliminar la violencia contra las mujeres, que parece ir en aumento tras las recientes revueltas en algunos paÃses de la regiÃ³n.",5
120,"Además, desde 2011, se ha estado trabajando para desarrollar marcos de recursos integrados locales para la atención de la salud y la comunidad, que permiten a los sistemas locales cuantificar el uso de recursos en la atención social y de la salud entre las poblaciones (en lugar de las organizaciones) y realinear los recursos en consecuencia. El objetivo es realinear los recursos para ofrecer un mejor valor y una mejor atención centrada en el paciente. Hay una clara innovación en el uso de la infraestructura digital para capacitar a los profesionales de la salud para que estén equipados con las habilidades acordes con las necesidades del NHS.",3
154,"También hay pagos relacionados con la colocación de personal de enfermería para la capacitación basada en estadías nocturnas para el apoyo a la independencia y la colocación de profesionales de la salud para los servicios de bienestar para discapacitados y hacen posible que las personas vivan en la comunidad mientras reciben apoyo relacionado con la salud. Se han establecido visitas de extensión de enfermería y servicios orientados a la comunidad en hospitales y clínicas privados para apoyar el alta temprana. También ha aumentado el número de guarderías psiquiátricas, clínicas ambulatorias psiquiátricas privadas e instalaciones de rehabilitación social. Además, el MHLW otorga subsidios para proyectos de “red de atención de salud mental infantil” implementados por la prefectura. Sin embargo, hay una escasez de hospitales centrales que se especialicen en la atención de la salud mental de los niños, con solo 21 hospitales con salas especializadas para niños y adolescentes en todo Japón. “Hikikomori”, un grupo identificado en Japón que generalmente se entiende como adolescentes solitarios o adultos jóvenes que se retiran de la vida social, a menudo buscando grados extremos de aislamiento y confinamiento, son motivo de especial preocupación.",3


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Textos_espanol  3000 non-null   object
 1   sdg             3000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 47.0+ KB


drop duplicated, dropna

In [33]:
############################################################
# Preprocesamiento Funciones: LIMPIEZA DE TEXTO
############################################################

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all integer occurrences in list of tokenized words with textual representation in Spanish"""
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = num2words(int(word), lang='es')
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in set(stopwords.words('spanish')):
            new_words.append(word)
    return new_words
def join_words(words):
    """Join a list of words into a single string"""
    return ' '.join(map(str, words))

def preprocessing(words):
    words = to_lowercase(words)
    words = replace_numbers(words)
    words = remove_punctuation(words)
    words = remove_non_ascii(words)
    words = remove_stopwords(words)
    words = join_words(words)
    return words

############################################################
# Preprocesamiento Funciones: LEMATIZACIÓN
############################################################
def stem_words(text):
    """Stem words in a text (Spanish)"""
    stemmer = SnowballStemmer("spanish")
    words = word_tokenize(text, language="spanish")
    stems = [stemmer.stem(word) for word in words]
    return stems
def lemmatize_verbs(text):
    """Lemmatize verbs in text (Spanish)"""
    nlp = spacy.load("es_core_news_sm")
    doc = nlp(text)
    lemmas = [token.lemma_ if token.pos_ == "VERB" else token.text for token in doc]
    return lemmas

def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems + lemmas

def lemmatize_all_words(text):
    """Lemmatize all words in a list (Spanish)"""
    nlp = spacy.load("es_core_news_sm")
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return join_words(lemmas)


In [57]:
data_petit = data.sample(20)
data_petit.reset_index(drop=True, inplace=True)

In [58]:
data_petit.head(1)

,Textos_espanol,sdg
0,"Dado que los participantes tuvieron un amplio grado de libertad para diseñar sus estrategias, la variabilidad permite extraer varias lecciones preliminares. Tras analizar las medidas de desarrollo profesional aplicadas por los equipos participantes en el proyecto, se destacan las principales lecciones extraídas de los planes de desarrollo profesional del profesorado aplicados por los equipos de los países para ayudar a los profesores participantes a fomentar la creatividad y el pensamiento crítico de sus alumnos. Como mínimo, los equipos debían presentar a los profesores los materiales del proyecto, los objetivos y los enfoques pedagógicos sugeridos organizando una sesión de formación inicial de un día de duración al inicio del proyecto.",4


In [51]:
data_petit.columns

Index(['Textos_espanol', 'sdg'], dtype='object')

In [59]:
data_petit['texto_limpio'] = data_petit['Textos_espanol'].swifter.apply(word_tokenize,language="spanish").swifter.apply(preprocessing)

Pandas Apply:   0%|          | 0/20 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/20 [00:00<?, ?it/s]

In [60]:
data_petit.head(1)

,Textos_espanol,sdg,texto_limpio
0,"Dado que los participantes tuvieron un amplio grado de libertad para diseñar sus estrategias, la variabilidad permite extraer varias lecciones preliminares. Tras analizar las medidas de desarrollo profesional aplicadas por los equipos participantes en el proyecto, se destacan las principales lecciones extraídas de los planes de desarrollo profesional del profesorado aplicados por los equipos de los países para ayudar a los profesores participantes a fomentar la creatividad y el pensamiento crítico de sus alumnos. Como mínimo, los equipos debían presentar a los profesores los materiales del proyecto, los objetivos y los enfoques pedagógicos sugeridos organizando una sesión de formación inicial de un día de duración al inicio del proyecto.",4,dado participantes amplio grado libertad disenar estrategias variabilidad permite extraer varias lecciones preliminares tras analizar medidas desarrollo profesional aplicadas equipos participantes proyecto destacan principales lecciones extraidas planes desarrollo profesional profesorado aplicados equipos paises ayudar profesores participantes fomentar creatividad pensamiento critico alumnos minimo equipos debian presentar profesores materiales proyecto objetivos enfoques pedagogicos sugeridos organizando sesion formacion inicial dia duracion inicio proyecto


In [61]:
data_petit['texto_limpio'] = data_petit['texto_limpio'].swifter.apply(stem_and_lemmatize).swifter.apply(join_words)
# data_petit['texto_limpio'] = data_petit['texto_limpio'].swifter.apply(lemmatize_all_words)

Pandas Apply:   0%|          | 0/20 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/20 [00:00<?, ?it/s]

In [62]:
data_petit.head(1)

,Textos_espanol,sdg,texto_limpio
0,"Dado que los participantes tuvieron un amplio grado de libertad para diseñar sus estrategias, la variabilidad permite extraer varias lecciones preliminares. Tras analizar las medidas de desarrollo profesional aplicadas por los equipos participantes en el proyecto, se destacan las principales lecciones extraídas de los planes de desarrollo profesional del profesorado aplicados por los equipos de los países para ayudar a los profesores participantes a fomentar la creatividad y el pensamiento crítico de sus alumnos. Como mínimo, los equipos debían presentar a los profesores los materiales del proyecto, los objetivos y los enfoques pedagógicos sugeridos organizando una sesión de formación inicial de un día de duración al inicio del proyecto.",4,dad particip ampli grad libert disen estrategi variabil permit extra vari leccion preliminar tras analiz med desarroll profesional aplic equip particip proyect destac principal leccion extra plan desarroll profesional profesor aplic equip pais ayud profesor particip foment creativ pensamient critic alumn minim equip debi present profesor material proyect objet enfoqu pedagog suger organiz sesion formacion inicial dia duracion inici proyect dar participantes amplio grado libertad disenar estrategias variabilidad permitir extraer varias lecciones preliminares tras analizar medidas desarrollo profesional aplicadas equipos participantes proyecto destacar principales lecciones extraidas planes desarrollo profesional profesorado aplicados equipos paises ayudar profesores participantes fomentar creatividad pensamiento critico alumnos minimo equipos debiar presentar profesores materiales proyecto objetivos enfoques pedagogicos sugeridos organizar sesion formacion inicial diar duracion inicio proyecto
